In [23]:
%%file td_utils.py
from datetime import datetime
from pytz import timezone

def format_history_quotes_result(res):
    if not res or ('empty' in res and res['empty']) or 'error' in res:
        return res
    candles = res['candles']
    for candle in candles:
        candle['datetime'] = datetime.fromtimestamp(candle['datetime'] / 1000.0, tz=timezone('America/New_York')).strftime('%Y-%m-%d %H:%M:%S')
    return res

def format_delayed_quote(res):
    if not res or 'error' in res:
        return res
    for k, v in res.items():
        v['quoteTime'] = datetime.fromtimestamp(v['quoteTimeInLong'] / 1000.0, tz=timezone('America/New_York')).strftime('%Y-%m-%d %H:%M:%S')
    return res

Overwriting td_utils.py


In [4]:
%%file td_api.py
import requests as r
import json
from datetime import timedelta
from td_utils import *

class TDBaseApi(object):
    def __init__(self, api_key):
        self.api_key = api_key
        
    def make_request(self, url, params=None):
        res = r.get(url, params=params).content.decode('utf-8')
        try:
            return json.loads(res, 'utf-8')
        except Exception:
            return res

class TDQuoteApi(TDBaseApi):
    GET_QUOTE_URL = 'https://api.tdameritrade.com/v1/marketdata/%s/quotes'
    GET_QUOTES_URL = 'https://api.tdameritrade.com/v1/marketdata/quotes'
    GET_HISTORY_QUOTE_URL = 'https://api.tdameritrade.com/v1/marketdata/%s/pricehistory'
    
    def get_delayed_quote(self, symbol):
        '''
        Get delayed quote with given symbol, case sensitive, eg: HUYA.
        '''
        
        if not isinstance(symbol, str):
            return None
        res = self.make_request(self.GET_QUOTE_URL % symbol, params={
            'apikey': self.api_key
        })
        return format_delayed_quote(res)
    
    def get_delayed_quotes(self, symbols):
        '''
        Get delayed quotes with given symbols, case sensitive, symbols are seperated by comma, eg: HUYA,IQ
        '''
        
        if not isinstance(symbols, str):
            return None
        return self.make_request(self.GET_QUOTES_URL, params={
            'apikey': self.api_key,
            'symbol': symbols
        })
    
    def get_history_quotes(self, symbol, period_type=None, period=None, frequency_type=None, frequency=None,
                          start_date=None, end_date=None, need_extended_hours_data=True):
        '''
        Get history quotes from given symbol and query params, case sensitive.
        
        :param period_type The type of period to show. Valid values are day, month, year, or ytd (year to date). Default is day.
        :param period The number of periods to show. Valid values are for day: 1, 2, 3, 4, 5, 10*, for month: 1*, 2, 3, 6 and for year: 1*, 2, 3, 5, 10, 15, 20
        :param frequency_type The type of frequency with which a new candle is formed. Valid values are minute, daily, weekly, monthly.
        :param frequency The number of the frequencyType to be included in each candle. For minute, valid values including 1, 5, 10, 15, 30.
        :param start_date Start date as milliseconds since epoch. If startDate and endDate are provided, period should not be provided.
        :param end_date End date as milliseconds since epoch. If startDate and endDate are provided, period should not be provided. Default is previous trading day.
        :param need_extended_hours_data true to return extended hours data, false for regular market hours only. Default is true
        '''
        if not isinstance(symbol, str):
            return None
        if isinstance(start_date, str):
            start_date = int(datetime.strptime(start_date, "%Y-%m-%d")
                             .replace(tzinfo=timezone('America/New_York')).timestamp() * 1000)
        if isinstance(end_date, str):
            end_date = int(datetime.strptime(end_date, "%Y-%m-%d")
                           .replace(tzinfo=timezone('America/New_York')).timestamp() * 1000)
        
        res = self.make_request(self.GET_HISTORY_QUOTE_URL % symbol, params={
            'apikey': self.api_key,
            'periodType': period_type,
            'period': period,
            'frequencyType': frequency_type,
            'frequency': frequency,
            'startDate': start_date,
            'endDate': end_date,
            'needExtendedHoursData': need_extended_hours_data
        })
        return format_history_quotes_result(res)
        
    
class TDTimeApi(TDBaseApi):
    GET_HOUR_URL = 'https://api.tdameritrade.com/v1/marketdata/%s/hours'
    
    def get_hour_for_single_market(self, market, date):
        '''
        Get valid market hour for a single market.
        
        :param market query market. Valid values are EQUITY, OPTION, FUTURE
        :param date "The date for which market hours information is requested. Valid ISO-8601 formats are : yyyy-MM-dd and yyyy-MM-dd'T'HH:mm:ssz."
        '''
        if not isinstance(market, str) or not isinstance(date, str):
            return None
        return self.make_request(self.GET_HOUR_URL % market, params={
            'apikey': self.api_key,
            'date': date
        })

Overwriting td_api.py


In [3]:
from td_api import *

In [9]:
api_key = 'HXSSG1124@AMER.OAUTHAP'
quote_api = TDQuoteApi(api_key)
time_api = TDTimeApi(api_key)
quotes = quote_api.get_history_quotes('HUYA', start_date=start_date, period=1, period_type='day', frequency_type='minute', frequency='', need_extended_hours_data=True)
#print(quotes)
if 'error' in quotes or not quotes:
    print(quotes)
else:
    candles = quotes['candles']
    print('\n'.join(list(map(lambda x: '%s,%s,%s' % (x['datetime'], x['close'], x['volume']), candles))))

2018-07-06 11:41:00,29.5478,6615
2018-07-06 11:42:00,29.63,11900
2018-07-06 11:43:00,29.6,2600
2018-07-06 11:44:00,29.66,7201
2018-07-06 11:45:00,29.65,1950
2018-07-06 11:46:00,29.62,10896
2018-07-06 11:47:00,29.59,9078
2018-07-06 11:48:00,29.52,14367
2018-07-06 11:49:00,29.53,37544
2018-07-06 11:50:00,29.5,14265
2018-07-06 11:51:00,29.55,9350
2018-07-06 11:52:00,29.5338,13310
2018-07-06 11:53:00,29.5,17410
2018-07-06 11:54:00,29.5368,8684
2018-07-06 11:55:00,29.5,15100
2018-07-06 11:56:00,29.5,4838
2018-07-06 11:57:00,29.47,11080
2018-07-06 11:58:00,29.5852,8105
2018-07-06 11:59:00,29.5867,11850
2018-07-06 12:00:00,29.6,7743
2018-07-06 12:01:00,29.6,5000
2018-07-06 12:02:00,29.65,1261
2018-07-06 12:03:00,29.626,5090
2018-07-06 12:04:00,29.5,10046
2018-07-06 12:05:00,29.53,8550
2018-07-06 12:06:00,29.71,37770
2018-07-06 12:07:00,29.77,2400
2018-07-06 12:08:00,29.7,10866
2018-07-06 12:09:00,29.7542,15105
2018-07-06 12:10:00,29.7056,17895
2018-07-06 12:11:00,29.65,13460
2018-07-06 12:12: